In [1]:
import pandas as pd
import numpy as np

#use this colab tool to upload files into colab notebook
# from google.colab import files
# file = files.upload()

# Dataset used here is the combination iof NSL-KDD train and test text files
#seting feature columns using numpy array

feature_col = np.arange(41)
train_ds = pd.read_csv("/content/drive/My Drive/RM3/KDDTotal.txt", usecols=(feature_col), dtype= str, encoding= "utf-8").values





In [3]:
train_ds

array([['0', 2, 44, ..., '0.00', '0.00', '0.00'],
       ['0', 1, 49, ..., '1.00', '0.00', '0.00'],
       ['0', 1, 24, ..., '0.01', '0.00', '0.01'],
       ...,
       ['0', 1, 24, ..., '0.00', '0.07', '0.07'],
       ['0', 2, 12, ..., '0.00', '0.00', '0.00'],
       ['0', 1, 57, ..., '0.00', '0.44', '1.00']], dtype=object)

In [4]:
#tools for preprocessing from sklearn

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score

# Encoding categeroical data 
# Note: Datasets used have 43 columns :Extra [:41] the 43rd column is attack rate which dont support the model 

# fit_transform is used to transform dataset after encoding

for c in range(1,4):
  train_ds[:,c] = LabelEncoder().fit_transform(train_ds[:,c])
train_ds[:,40] = LabelEncoder().fit_transform(train_ds[:,40])
print(train_ds)
# Now all features are in numeric 

[['0' 2 44 ... '0.00' '0.00' 0]
 ['0' 1 49 ... '1.00' '0.00' 0]
 ['0' 1 24 ... '0.01' '0.00' 1]
 ...
 ['0' 1 24 ... '0.00' '0.07' 7]
 ['0' 2 12 ... '0.00' '0.00' 0]
 ['0' 1 57 ... '0.00' '0.44' 100]]


In [6]:
# Here all numeric features converted into Float

train_ds=train_ds.astype(np.float)

# Here we standardizing training features

# sc = StandardScaler()
# sc.fit(train_ds)
# ds_normalized = sc.transform(train_ds)

# MinMax scalling 

MMS = MinMaxScaler(feature_range = (0,1))
MMS.fit(train_ds)

# target column for training is taken from the dataset

target_tn=pd.read_csv("/content/drive/My Drive/RM3/KDDTotal.txt", usecols=([41]), dtype= str, encoding= "utf-8").values

# As dataset contains #many number of target features
# For the usability in NIDS it should be able to recognize kind of traffic
# To classify the traffic target labels to be as anomaly or normal traffic

for i in range(0,len(target_tn)):
  if target_tn[i] != 'normal':
    target_tn[i] = 'anomaly'
target_tn

#Creating training pipeline

ds_normalized = MMS.transform(train_ds)
target_label = LabelEncoder().fit_transform(target_tn)

# Feature engineering of training data is finished



/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [7]:
# Splitting test, train pipelines with features(x), target(y)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(ds_normalized, target_label, test_size=.5, random_state=40)


In [14]:
# SVM algorithm is available in sklearn
import time
from sklearn import svm
from matplotlib import  pyplot as plt
# SVC classifier with various parameters is fitted with input/output pielines

clf = svm.SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3,
     kernel= 'poly', max_iter=-1, probability=False, decision_function_shape = 'ovo',
    random_state=None, shrinking=True, tol=0.001, verbose=False)

start = time.time()
clf.fit(x_train, y_train)
stop = time.time()
plt.clf()
print(f"training time: {stop- start}s")
# predicting accuracy of model with testing data

predictions = clf.predict(x_test)

# scoring overall trained model on new data

from sklearn import metrics
score = metrics.accuracy_score(y_test, predictions)
print("score of SVC is:", score)

training time: 38.17621326446533s
score of SVC is: 0.971639419321824


<Figure size 432x288 with 0 Axes>

In [ ]:
#linear

training time: 77.69244647026062s
score of SVC is: 0.9339061111260739